<a href="https://colab.research.google.com/github/yuta4u2/test1/blob/master/GrowthAccounting_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


# Load data
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
countries = ["France", "Germany", "Canada", "Italy", "Japan", "United Kingdom", "United States"] # add later 
data = pwt1001.loc[pwt1001['country'].isin(countries)][['year', 'countrycode', 'rgdpna', 'rkna', 'pop', 'emp', 'labsh', 'avh']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# Calculate additional variables
data['alpha'] = 1 - data['labsh']
data['hours'] = data['emp'] * data['avh']
data['y_L'] = np.log(data['rgdpna'] / data['hours'])  # ln(Y/L)
data['k_L'] = np.log(data['rkna'] / data['hours'])  # ln(K/L)
data['tfp'] = np.log(data['rtfpna']) # ln(A)

# Order by year
data = data.sort_values("year")

# Group by isocode
grouped_data = data.groupby('countrycode')

# Calculate growth rates 
data["Growth_rate"] = grouped_data['y_L'].diff() * 100
data["Capital_Growth"] = grouped_data['k_L'].diff() * 100
data["Capital_Deepning"] = data["Capital_Growth"] * data['alpha']
data["TFP_Growth"] = grouped_data['tfp'].diff() * 100


data = data.dropna()

summary = data.groupby('countrycode').agg({'Growth_rate': 'mean',
                                       'Capital_Growth': "mean",
                                       'TFP_Growth': "mean"})
summary['TFP_Share'] = summary['TFP_Growth']/summary['Growth_rate']
summary['Capital_Share'] = summary['Capital_Deepning']/summary['Growth_rate']
summary = summary.reindex(columns=['Growth_rate', 'TFP_Growth', 'Capital_Deepning', 'TFP_Share', 'Capital_Share'])
summary.loc['Average'] = summary.mean()
summary

,Growth_rate,Capital_Growth,TFP_Growth,Capital Share,Capital_Deepning
countrycode,,,,,
CAN,1.117384,2.065398,1.677696,0.346190,0.715021
DEU,1.073874,1.611113,1.409886,0.370730,0.597288
FRA,1.087678,1.750044,0.247013,0.381194,0.667107
GBR,1.149943,1.514244,-1.598219,0.412993,0.625372
ITA,0.285631,1.426465,-0.256090,0.484835,0.691599
JPN,1.221666,2.016915,2.118842,0.430601,0.868486
USA,1.658237,2.135963,1.355163,0.393260,0.839989


In [ ]:
data.head()

,year,countrycode,rgdpna,rkna,pop,emp,labsh,rtfpna,Capital Share,y_emp,k_emp,tfp,Growth_rate,Capital_Growth,TFP_Growth
5926,1996,ITA,2.170005e+06,0.689952,57.065225,22.114836,0.527412,1.181289,0.472588,11.493992,-3.467383,13.132636,0.867462,2.107334,-0.827026
4316,1996,GBR,1.885852e+06,0.604429,58.113557,25.924555,0.533988,0.936591,0.466012,11.194699,-3.758662,12.946282,1.473427,2.709079,4.085636
12156,1996,USA,1.188185e+07,0.539946,268.335003,129.357666,0.607116,0.872310,0.392884,11.427941,-5.478868,13.580500,2.227402,2.413130,1.423073
4176,1996,FRA,2.043539e+06,0.619433,59.748767,23.806284,0.623363,0.973417,0.376637,11.360244,-3.648901,12.734555,0.780487,1.626468,-1.006889
6136,1996,JPN,4.285729e+06,0.719400,126.644094,67.725853,0.589143,0.974287,0.410857,11.055333,-4.544806,12.922597,2.952003,4.471302,5.530548
